# Part I. ETL Pipeline for Pre-Processing the Files

The project template and data can be found at Udacity - Topic: Data Engineering with AWS, course 2, lesson 4.

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import os
import glob
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="image_event_datafile_new.jpg">

#### Starting Cassandra Server Locally with Docker

To quickly set up a Cassandra server locally, with your Docker opened, please conduct these provided commands in your terminal:

```bash
# Pull the latest Cassandra Docker image
docker pull cassandra:latest

# Create a Docker network for Cassandra
docker network create cassandra

# Run the Cassandra container
docker run --rm -d --name cassandra --hostname cassandra --network cassandra -p 9042:9042 cassandra

# Verify that the container is running
docker ps

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
# Connect to a Cassandra instance on local machine
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, we need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS data_proj
                    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set Keyspace
try:
    session.set_keyspace('data_proj')
except Exception as e:
    print(e)

Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

The purpose of this notebook is to create queries that help answer the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Let's first see our data created in part 1

In [8]:
pd.read_csv('event_datafile_new.csv').head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26
2,Train,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Marry Me,26
3,Sony Wonder,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",597,Blackbird,61
4,Van Halen,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",602,Best Of Both Worlds (Remastered Album Version),80


#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

To answer this question, firstly, I will create a song_info table containing all the required columns: sessionId, itemInSession, artist, song, length. Then, I choose sessionId as the partition key, itemInSession as the clustering column, together I have a unique primary key including sessionId and itemInSession.

In [9]:
# Create song_info table
query1 = "CREATE TABLE IF NOT EXISTS song_info"
query1 += "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query1)
    print("Table 'song_info' created successfully.")
except Exception as e:
    print("Error creating table 'song_info':", e)

Table 'song_info' created successfully.


In [10]:
# Insert data into song_info table
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query1 = "INSERT INTO song_info (sessionId, itemInSession, artist, song, length)"
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
    f.close()

In [11]:
# Conduct query to answer question 1
query1 = "SELECT artist, song, length FROM song_info WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

for row in rows:
    print(f"Artist: {row.artist}, Song: {row.song}, Length: {row.length}")

Artist: Faithless, Song: Music Matters (Mark Knight Dub), Length: 495.30731201171875


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

To answer this question, firstly, I will create a song_user_info table containing all the required columns: userId, sessionId, itemInSession, artist, song, firstName, lastName. Then, I choose userId and sessionId as the composite partition key, and itemInSession as the clustering column, together I have a unique primary key including userId, sessionId and itemInSession.

In [12]:
# Create song_user_info table
query2 = "CREATE TABLE IF NOT EXISTS song_user_info"
query2 += "(userId int, sessionId int, itemInSession int, artist text, song text, \
           firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession)) \
           WITH CLUSTERING ORDER BY (itemInSession ASC)"

try:
    session.execute(query2)
    print("Table 'song_user_info' created successfully.")
except Exception as e:
    print("Error creating table 'song_user_info':", e)                    

Table 'song_user_info' created successfully.


In [13]:
# Insert data into song_user_info table
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query2 = "INSERT INTO song_user_info (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query2 = query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
    f.close()

In [14]:
# Conduct query to answer question 2
query2 = "SELECT artist, song, firstName, lastName FROM song_user_info WHERE userId = 10 AND sessionId = 182"

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

for row in rows:
    print(f"Artist: {row.artist}, Song: {row.song}, First Name: {row.firstname}, Last Name: {row.lastname}")

Artist: Down To The Bone, Song: Keep On Keepin' On, First Name: Sylvie, Last Name: Cruz
Artist: Three Drives, Song: Greece 2000, First Name: Sylvie, Last Name: Cruz
Artist: Sebastien Tellier, Song: Kilometer, First Name: Sylvie, Last Name: Cruz
Artist: Lonnie Gordon, Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), First Name: Sylvie, Last Name: Cruz


#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
To answer this question, firstly, I will create a user_info table containing all the required columns: userId, song, firstName, lastName. Then, I choose song as the partition key, and userId as the clustering column, together I have a unique primary key including song and userId.

In [15]:
# Create user_info table
query3 = "CREATE TABLE IF NOT EXISTS user_info"
query3 += "(userId int, song text, firstName text, lastName text, PRIMARY KEY (song, userId))"

try:
    session.execute(query3)
    print("Table 'user_info' created successfully.")
except Exception as e:
    print("Error creating table 'user_info':", e)                    

Table 'user_info' created successfully.


In [16]:
# Insert data into user_info table
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query3 = "INSERT INTO user_info (userId, song, firstName, lastName)"
        query3 = query3 + " VALUES (%s, %s, %s, %s)"
        session.execute(query3, (int(line[10]), line[9], line[1], line[4]))
    f.close()

In [17]:
# Conduct query to answer question 3
query3 = "SELECT firstName, lastName FROM user_info WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

for row in rows:
    print(f"First Name: {row.firstname}, Last Name: {row.lastname}")

First Name: Jacqueline, Last Name: Lynch
First Name: Tegan, Last Name: Levine
First Name: Sara, Last Name: Johnson


#### Drop the tables before closing out the sessions

In [18]:
try:
    session.execute("DROP TABLE IF EXISTS song_info")
    print("Dropped table 'song_info' successfully.")
    session.execute("DROP TABLE IF EXISTS song_user_info")
    print("")
    print("Dropped table 'song_user_info' successfully.")
    session.execute("DROP TABLE IF EXISTS user_info")
    print("")
    print("Dropped table 'user_info' successfully.")
except Exception as e:
    print(e)

Dropped table 'song_info' successfully.

Dropped table 'song_user_info' successfully.

Dropped table 'user_info' successfully.


#### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()